In [ ]:
%reset -f

In [ ]:
import os
os.chdir(r'/home')
import numpy as np
import scipy
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

# GPU
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

# Functions
def derivee(y,x):
  return torch.autograd.grad(y,x,torch.ones_like(y),retain_graph=True,create_graph=True)[0]

def som(s):
  return torch.sum(s,dim=-1,keepdim=True)

namedata = 'Struct_1_512' #
Data = scipy.io.loadmat(namedata+'.mat')
pix = torch.tensor(Data['pixels'], dtype = torch.float32).to(device)

In [ ]:
# NN module
class MLP_gelu(nn.Module):
    def __init__(self,insize,outsize,width,layers):
        super().__init__()
        self.input_layer = nn.Linear(insize,width)
        self.hidden_layers = nn.ModuleList([])
        self.output_layer = nn.Linear(width,outsize)
        for i in range(layers):
          self.linear_layer = nn.Linear(width,width)
          self.hidden_layers.append(self.linear_layer)
        self.layers = layers

    def forward(self, x):
        layer_out = F.gelu(self.input_layer(x))
        for i in range(self.layers):
            layer_out = F.gelu(self.hidden_layers[i](layer_out)) # + layer_out
        layer_out = self.output_layer(layer_out)
        return layer_out

# Neural network in PINN
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.NN_trunk = MLP_gelu(insize=4,outsize=2,width=64,layers=3)

    def forward(self, x, y, e):
        # Normalisation
        scaler = torch.max(torch.abs(e))
        e = e/scaler
        # NN modulus
        U_tilde = self.NN_trunk(torch.cat([torch.sin(x * 2*torch.pi),
                                     torch.cos(x * 2*torch.pi),
                                     torch.sin(y * 2*torch.pi),
                                     torch.cos(y * 2*torch.pi)], dim=-1))
        ux_tilde,uy_tilde = U_tilde[...,0:1],U_tilde[...,1:2]
        # Periodic Boundary condition
        ux = (ux_tilde + e[0]*x + e[2]*y)*scaler
        uy = (uy_tilde + e[2]*x + e[1]*y)*scaler
        return ux, uy

class Homogenisation():
    def __init__(self, struct, epsilon_bar):
        super(Homogenisation, self).__init__()

        # Configuration
        self.struct = struct
        # Average Strain
        self.epsilon_bar = epsilon_bar
        # Constitutive Model: hyperelasticity parameters
        self.C10, self.D1 = 1.93, 0.24
        # Solution (Displacement)
        self.U = PINN().to(device)
        # Coordinates
        Ndim = len(struct)
        X, Y = torch.meshgrid(torch.linspace(-0.5+0.5/Ndim,0.5-0.5/Ndim,Ndim),
                              torch.linspace(-0.5+0.5/Ndim,0.5-0.5/Ndim,Ndim),
                              indexing='ij')
        L_pde = (self.struct.reshape(-1) == 1).to(device)
        self.x_pde = X.reshape(-1)[:,None].to(device)[L_pde]
        self.y_pde = Y.reshape(-1)[:,None].to(device)[L_pde]
        self.x_pde.requires_grad = True
        self.y_pde.requires_grad = True
        # Iteration
        self.iter = 0
        self.scaler = torch.max(torch.abs(epsilon_bar))

    def Psi(self): # strain energy (Neo-Hookean model)
        # Displacement
        ux, uy = self.U(self.x_pde, self.y_pde, self.epsilon_bar)
        # Deformation Gradient
        F = torch.zeros(len(self.x_pde), 2, 2, dtype=ux.dtype, device=ux.device)
        F[:, 0, 0] = derivee(ux, self.x_pde).squeeze() + 1.0  # 1 + dux/dx
        F[:, 1, 1] = derivee(uy, self.y_pde).squeeze() + 1.0  # 1 + duy/dy
        F[:, 0, 1] = derivee(ux, self.y_pde).squeeze() # dux/dy
        F[:, 1, 0] = derivee(uy, self.x_pde).squeeze() # duy/dx
        # Jacobian determinant
        J = F[:, 0, 0] * F[:, 1, 1] - F[:, 0, 1] * F[:, 1, 0]
        J = J.clamp(min=1e-8)  # avoid log(0) or negative roots
        # Left Cauchy-Green tensor B = F * F^T
        B  = F @ F.transpose(-1, -2) # (N,2,2)
        I1_in_plane = B.diagonal(dim1=-2, dim2=-1).sum(-1)   # B11 + B22
        I1 = I1_in_plane + 1.0 # +  B33  (plane-strain ⇒ F33 = 1)
        I1_bar = J.pow(-2.0/3.0) * I1
        # Neo-Hookean deformation density function
        psi = self.C10 * (I1_bar - 3.0) + (1.0 / self.D1) * (J - 1.0).pow(2)
        # scaling
        psi = psi / self.scaler**2 # loss scaling
        return psi.mean()

    def Solution(self): # Calculate stress field
        # Displacement
        ux, uy = self.U(self.x_pde, self.y_pde, self.epsilon_bar)
        U = torch.cat((ux,uy),dim=-1)
        U = U-torch.mean(U,dim=0,keepdim=True)
        # Deformation Gradient
        F = torch.zeros(len(self.x_pde), 2, 2, dtype=ux.dtype, device=ux.device)
        F[:, 0, 0] = derivee(ux, self.x_pde).squeeze() + 1.0
        F[:, 1, 1] = derivee(uy, self.y_pde).squeeze() + 1.0
        F[:, 0, 1] = derivee(ux, self.y_pde).squeeze()
        F[:, 1, 0] = derivee(uy, self.x_pde).squeeze()
        # Jacobian determinant
        J = (F[:, 0, 0] * F[:, 1, 1] - F[:, 0, 1] * F[:, 1, 0]).clamp(min=1e-8)
        F_inv_T = torch.linalg.inv(F).transpose(-1, -2)
        # Invariants
        B   = F @ F.transpose(-1, -2)
        I1  = B.diagonal(dim1=-2, dim2=-1).sum(-1) + 1.0# +B33 (plane-strain)
        Jm23   = J.pow(-2.0 / 3.0)
        I1_bar = Jm23 * I1 # (N,)
        # first-Piola stress
        # iso-choric part
        Jm23_ = Jm23.view(-1, 1, 1)  # (N,1,1) for broadcast
        I1b_  = I1_bar.view(-1, 1, 1)
        iso_term = 2.0 * self.C10 * ( Jm23_ * F - (1.0/3.0) * I1b_ * F_inv_T )
        # volumetric part
        vol_coeff = ( (J - 1.0) * J * 2.0 / self.D1 ).view(-1, 1, 1)
        vol_term  = vol_coeff * F_inv_T
        # first-Piola stress
        P = iso_term + vol_term
        return U,F,P

    def closure(self): # closure function for L-BSGF optimisor
        self.iter += 1
        optimizer.zero_grad()
        Loss = self.Psi()
        Loss.backward()
        if (self.iter+1)%100==1:
            print(f'Epoch:{self.iter}, PDE Loss:{Loss:.3e}')
        return Loss

In [ ]:
Macro_Epsilon = torch.tensor([0.0,-0.5,0.0]).to(device)
model = Homogenisation(pix,Macro_Epsilon)

lr = 1.0
optimizer = torch.optim.LBFGS(model.U.parameters(),lr=lr,
                              max_iter=2000,
                              max_eval=None,
                              tolerance_grad=0,
                              tolerance_change=0,
                              history_size=100)

optimizer.step(model.closure)

In [ ]:
model.U.eval()
U, Epsilon, Sigma = model.Solution()

ux, uy = U[:, 0:1], U[:, 1:2]
x = (model.x_pde + ux).detach().cpu().numpy()
y = (model.y_pde + uy).detach().cpu().numpy()
ux = ux.detach().cpu().numpy()
uy = uy.detach().cpu().numpy()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 2.5))
ax1.set_title('u_x'); ax1.axis('off'); ax1.axis('equal')
ax2.set_title('u_y'); ax2.axis('off'); ax2.axis('equal')
plt.colorbar(ax1.scatter(x, y, c=ux, cmap='jet', s=0.1), ax=ax1)
plt.colorbar(ax2.scatter(x, y, c=uy, cmap='jet', s=0.1), ax=ax2)
plt.tight_layout()
plt.show()